In [1]:
import numpy as np
import pandas as pd
import os
import warnings
import librosa
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from random import shuffle
import scipy.stats as st

from pynwb import NWBHDF5IO
from pynwb.image import ImageSeries
from pynwb.epoch import TimeIntervals

In [2]:
# ----- User defined parameters -----
nwbpath = 'M:\\scn2a-paper-GWJSNH\\NWB-files'
usefield = 'annotated_behavior'
writeToDisk = False

In [3]:
# ----- File handling -----
allnwb = []
allnwb += [os.path.join(nwbpath,each) for each in os.listdir(nwbpath) if each.endswith('.nwb')]

In [5]:
allnwb
test = [allnwb[0]]

In [17]:
# ----- Get all times, durations of requested behavior -----

# loop over files
for i,fname in enumerate(test):
    # open file
    io = NWBHDF5IO(os.path.join(nwbpath,fname), mode="r") # <-- open in write to append snippets?
    nwbfile = io.read()
    
    pathsplit = fname.split('\\')
    recstem = pathsplit[-1].strip('.nwb')
    
    print(recstem)
       
    # get aggregated annotation table
    statsdf = nwbfile.intervals[usefield].to_dataframe()
    
    # find start time of assay
    assayStart = float(statsdf.start_time[statsdf.behavior=='Start assay'])
    
    # get a list of all behaviors
    allbehav = np.unique(statsdf.behavior)
    
    # remove the start assay code
    behav = []
    behav += [each for each in allbehav if each != 'Start assay']
    
    # loop over each behavior to make a separated table
    for bhv in behav:
        print(f'Isolating behavior {bhv}')
        thisb = statsdf[statsdf.behavior==bhv]

        # make a table of start_time, stop_time, duration labeled with the behavior annotation
        thisb.start_time = thisb.start_time-assayStart
        thisb.stop_time = thisb.stop_time-assayStart

        # save that back to the NWB file
        
            # handle PPTs
        if nwbfile.lab_meta_data['vole_metadata'].assay_type == 'PPT':
            pchamb = nwbfile.lab_meta_data['vole_metadata'].assay_type__partner_chamber
            
            # FIX THIS TO LABEL BY PARTNER AND STRANGER CHAMBER
            
            if pchamb in bhv:
                
        
        this_behav_table = TimeIntervals.from_dataframe(thisb,f'single_behavior_{bhv}',
                                          table_description=f'Assay-aligned annotations for behavior coded {bhv}.')

        # add data
        nwbfile.add_time_intervals(this_behav_table)
        
    if writeToDisk:    
        io.write(nwbfile)
    
    io.close()

Nov22_Pair15_aggressio
Isolating behavior Aggression receipt
Isolating behavior Defensive rear
Isolating behavior Defensive strike
Isolating behavior Huddle
Isolating behavior Investigate
Isolating behavior No interaction
Isolating behavior Sniff
Isolating behavior Strike
Isolating behavior Tussle


C:\Users\Nerissa\AppData\Local\Temp\ipykernel_13004\1324250382.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thisb.start_time = thisb.start_time-assayStart
C:\Users\Nerissa\AppData\Local\Temp\ipykernel_13004\1324250382.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thisb.stop_time = thisb.stop_time-assayStart


In [24]:
nwbfile.lab_meta_data['vole_metadata'].assay_type__days_post_pairing

8

In [9]:
allbehav

array(['Aggression receipt', 'Defensive rear', 'Defensive strike',
       'Huddle', 'Investigate', 'No interaction', 'Sniff', 'Start assay',
       'Strike', 'Tussle'], dtype=object)

In [ ]:
# ----- Notes -----
# when I want something like counts and latencies, I can just ask for the size and first instance from those tables
# they will be easy to feed into rasters and histograms